In [ ]:
"""
Посчитайте выручку (Revenue) по странам и отсортируйте таблицу в порядке убывания выручки. В качестве ответа скопируйте
название страны с самой высокой выручкой.  Обратите внимание, что в изначальной таблице указана цена одной единицы
товара (UnitPrice) и количество купленного товара (Quantity), а не выручка, и для получения Revenue нужно будет использовать
агрегирующую функцию Sum и оператор умножения *.

SELECT
    Country,
    SUM(UnitPrice*Quantity) AS Revenue
FROM
    retail
GROUP BY
    Country
ORDER BY
    Revenue DESC
LIMIT 5
"""

In [ ]:
"""
Давайте посмотрим на динамику выручки от покупателей, которые в среднем покупают самые дорогие товары. 
В качестве целевой метрики будем использовать среднюю цену купленного товара (UnitPrice), данные посмотрим за март 2011 года.

Посчитайте среднюю цену товара клиента (UnitPrice) с группировкой по id покупателя, используя агрегирующую функцию Avg,
затем выведите строки с покупками тех покупателей, у которых самая высокая средняя цена купленного товара в марте. Не забывайте
отфильтровать строки по условию Description != 'Manual'. В качестве ответа впишите CustomerID покупателя с самой высокой
средней ценой купленного товара.

SELECT
    CustomerID,
    AVG(UnitPrice) AS Revenue
FROM
    retail
WHERE
        Description != 'Manual'
    AND
        toStartOfMonth(InvoiceDate) = '2011-03-01'
GROUP BY
    CustomerID
ORDER BY
    Revenue DESC
LIMIT 5
"""

# JOIN 

In [ ]:
"""
Давайте с помощью JOIN дополним таблицу events (left) данными о UserID пользователей из таблицы devices (right). 
Для некоторых DeviceID не будет пары UserID из таблицы devices – подумайте, какой вид JOIN подойдет, чтобы не потерять 
те строки, где DeviceID есть в events, но нет в devices.

Укажите UserID из первой строки результирующей таблицы, используя сортировку по убыванию по полю DeviceID.

SELECT
    l.DeviceID AS DeviceID,
    r.UserID
FROM 
    events AS l
LEFT JOIN devices AS r
    ON l.DeviceID = r.DeviceID
ORDER BY
    DeviceID DESC
LIMIT 100
"""

In [ ]:
"""
Давайте проверим, пользователи пришедшие из какого источника совершили наибольшее число покупок. В качестве ответа
выберите название Source, юзеры которого совершили больше всего покупок.

SELECT
    a.Source AS Source,
    COUNT(UserID) AS purchases
FROM 
    installs AS a
JOIN
    (
    SELECT
        c.UserID AS UserID,
        b.DeviceID AS DeviceID
    FROM devices AS b
    JOIN checks AS c
    ON b.UserID = c.UserID
    ) AS b
ON a.DeviceID = b.DeviceID
GROUP BY
    Source
ORDER BY 
    purchases DESC
LIMIT 100
"""

In [ ]:
"""
Выведите идентификаторы устройств пользователей, которые совершили как минимум одну покупку за последний месяц (октябрь 2019).
Используйте сортировку по возрастанию DeviceID и укажите минимальное значение.

SELECT
    b.DeviceID AS DeviceID
FROM
    checks AS a
JOIN
    devices AS b
ON a.UserID = b.UserID
WHERE 
    toStartOfMonth(Date(BuyDate)) == '2019-10-01'
ORDER BY
    DeviceID DESC
LIMIT 100
"""

In [ ]:
"""
Давайте посчитаем число уникальных DeviceID в инсталлах, для которых присутствуют просмотры в таблице events с разбивкой
по платформам (поле Platform). Для этого можно отобрать все строки только из таблицы installs, для которых нашлось соответствие
в таблице events. 

В качестве ответа введите число инсталлов, для которых нашлись просмотры, на платформе android.

SELECT
    Platform,
    uniqExact(b.DeviceID) AS NumUniqDevice
FROM
    events AS a
LEFT JOIN
    installs AS b
ON a.DeviceID = b.DeviceID
WHERE 
    Platform = 'android'
GROUP by
    Platform
"""

In [ ]:
"""
Давайте теперь посчитаем конверсию из инсталла в просмотр с разбивкой по платформе инсталла – в данном случае это доля DeviceID,
для которых есть просмотры, от всех DeviceID в инсталлах. 

SELECT
    Platform,
    COUNT(DISTINCT b.DeviceID) / COUNT(DISTINCT a.DeviceID)
FROM
    installs AS a
LEFT JOIN
    events AS b
ON a.DeviceID = b.DeviceID
GROUP BY
    Platform
"""

In [ ]:
"""
Выведите 10 уникальных DeviceID, которые присутствуют в таблице events,
но отсутствуют в installs, отсортировав их в порядке убывания. В качестве ответа введите первый DeviceID из списка.  

SELECT
    DISTINCT(a.DeviceID) AS UniqDeviceID,
    b.DeviceID
FROM
    events AS a
LEFT JOIN
    installs AS b
ON a.DeviceID = b.DeviceID
WHERE b.DeviceID = 0
ORDER BY
    UniqDeviceID DESC
LIMIT 100
"""

# ТИПЫ ДАННЫХ

In [ ]:
"""
Давайте проверим, в какой месяц и год зарегистрировалось наибольшее количество новых хостов.

SELECT
    toStartOfMonth(toDateOrNull(host_since)) AS date_registered,
    COUNT(host_id) AS users
FROM listings
GROUP BY date_registered
ORDER BY users DESC
LIMIT 3
"""

In [ ]:
"""
Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t).

SELECT
    host_is_superhost,
    AVG(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) AS avg_rate_host
FROM listings
GROUP BY host_is_superhost
ORDER BY avg_rate_host DESC
LIMIT 3
"""

In [ ]:
"""
Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает
(у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу
отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). В качестве ответа укажите первый массив в
результирующей таблице, состоящий более чем из двух id. Например:

SELECT
    host_id,
    groupArray(id),
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price
FROM listings
GROUP BY host_id
ORDER BY avg_price DESC, host_id DESC
LIMIT 100
"""

In [ ]:
"""
Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина.
В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей. 

SELECT
    host_id,
    groupArray(id),
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
    MAX(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) - MIN(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS diff_price
FROM listings
GROUP BY host_id
ORDER BY diff_price DESC
LIMIT 100
"""

In [ ]:
"""
Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки. Обратите 
внимание на тип данных, наличие значка $ и запятых в больших суммах. Для какого типа жилья среднее значение залога наибольшее?

SELECT
    room_type,
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
    AVG(toFloat32OrNull(replaceRegexpAll(security_deposit, '[$,]', ''))) AS avg_deposit,
    AVG(toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))) AS avg_cleaning_fee
FROM listings
GROUP BY room_type
LIMIT 100
"""

In [ ]:
"""
Отлично! В каких частях города средняя стоимость за ночь является наиболее низкой? 

SELECT
    neighbourhood_cleansed,
    AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price
FROM listings
GROUP BY neighbourhood_cleansed
ORDER BY avg_price ASC
LIMIT 100
"""

In [ ]:
"""
В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? Отсортируйте по среднему и 
выберите топ-3. 

SELECT
    neighbourhood_cleansed,
    AVG(toFloat32OrNull(replaceRegexpAll(square_feet, '[$,]', ''))) AS avg_price
FROM listings
WHERE room_type = 'Entire home/apt'
GROUP BY neighbourhood_cleansed
ORDER BY avg_price DESC
LIMIT 100
"""

In [ ]:
"""
Напоследок давайте посмотрим, какая из представленных комнат расположена ближе всего к центру города. В качестве ответа 
укажите id объявления.

SELECT
    id,
    host_id,
    toFloat64OrNull(longitude) AS longitude,
    toFloat64OrNull(latitude) AS latitude,
    geoDistance(13.4050, 52.5200, longitude, latitude) AS distance
FROM listings
WHERE room_type = 'Private room'
ORDER BY distance ASC
LIMIT 100
"""

# ПОДЗАПРОСЫ, WITH, VIEW

In [ ]:
"""
Сначала оставьте только те объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет 
строго меньше трёх. Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке. В качестве 
ответа укажите id объявления из первой строки.

SELECT
    id
FROM listings
WHERE
        toFloat32OrNull(review_scores_rating) > (SELECT AVG(toFloat32OrNull(review_scores_rating)) FROM listings)
    AND
        reviews_per_month < 3
ORDER BY reviews_per_month DESC, (toFloat32OrNull(review_scores_rating)) DESC
LIMIT 100
"""

In [ ]:
"""
Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых 
расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее 
удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем. 

WITH 
    (SELECT 
        AVG(geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)))
    FROM listings
    WHERE room_type == 'Private room') AS avg_distance


SELECT
    host_id,
    geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)) AS distance
FROM listings
WHERE distance < avg_distance
ORDER BY distance DESC
LIMIT 100
"""

In [ ]:
"""
В этой задаче можно потренироваться в написании подзапросов, но задание можно решить и без них.

Отберите объявления из таблицы listings, которые:

находятся на расстоянии от центра меньше среднего (здесь нам пригодится запрос из предыдущего задания)
обойдутся дешевле 100$ в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю, т.е его нужно
делить на кол-во дней)
имеют последние отзывы (last_review), начиная с 1 сентября 2018 года
имеют WiFi в списке удобств (amenities)

WITH 
    (SELECT 
        AVG(geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)))
    FROM listings) AS avg_distance


SELECT
    host_id,
    (toFloat32OrNull(replaceRegexpAll(price, '[$,]', '')) + toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))) / 7 AS full_price,
    toDateOrNull(last_review) AS last_review,
    geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)) AS distance
FROM listings
WHERE
        distance < avg_distance
    AND
        multiSearchAnyCaseInsensitive(amenities, ['WiFi'])
    AND
        last_review >= '2018-09-01'
    AND full_price < 100
ORDER BY toFloat32OrNull(review_scores_rating) DESC
LIMIT 100
"""

In [ ]:
"""
Давайте найдем в таблице calendar_summary те доступные (available='t') объявления, у которых число отзывов от уникальных 
пользователей в таблице reviews выше среднего.

WITH 
    (SELECT
        AVG(reviewer)
    FROM
        (
        SELECT 
            COUNT(DISTINCT reviewer_id) AS reviewer
        FROM
            reviews
        GROUP BY listing_id
        )
    ) AS reviewers
    

SELECT 
    l.listing_id,
    COUNT(DISTINCT l.reviewer_id) AS review_num
FROM
    reviews AS l
JOIN
    calendar_summary AS r
ON l.listing_id = r.listing_id
WHERE 
    r.available='t'
GROUP BY l.listing_id
HAVING
        review_num > reviewers
ORDER BY l.listing_id ASC
LIMIT 10
"""

In [ ]:
"""
Теперь задача! Используйте таблицу checks и разделите всех покупателей на сегменты:

NB! Правые границы берутся не включительно, например, чек в 10 рублей будет относиться к сегменту С

А — средний чек покупателя менее 5 ₽
B — средний чек покупателя от 5-10 ₽
C — средний чек покупателя от 10-20 ₽
D — средний чек покупателя от 20 ₽
Отсортируйте результирующую таблицу по возрастанию UserID и укажите сегмент четвертого пользователя.

SELECT
    UserID,
    CASE
        WHEN AVG(Rub) < 5 THEN 'A'
        WHEN 5 >= AVG(Rub) AND AVG(Rub) < 10 THEN 'B'
        WHEN 10 >= AVG(Rub) AND AVG(Rub) < 20 THEN 'C'
        ELSE 'D'
    END AS segment
FROM checks
GROUP BY UserID
ORDER BY UserID ASC
LIMIT 100
"""

In [ ]:
"""
Вернемся к таблице AirBnb. Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (kitchen) и гибкой 
системы отмены (flexible), причем первый в приоритете.

Создайте с помощью оператора CASE колонку с обозначением группы, в которую попадает жилье из таблицы listings:

'good', если в удобствах (amenities) присутствует кухня и система отмены (cancellation_policy) гибкая
'ok', если в удобствах есть кухня, но система отмены не гибкая
'not ok' во всех остальных случаях
Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, в качестве ответа укажите host_id 
первой строки.

SELECT
    host_id,
    CASE
        WHEN (multiSearchAnyCaseInsensitive(amenities, ['kitchen']) AND cancellation_policy == 'flexible') THEN 'good'
        WHEN (multiSearchAnyCaseInsensitive(amenities, ['kitchen']) AND cancellation_policy != 'flexible') THEN 'ok'
        ELSE 'not ok'
    END AS segment
FROM
    listings
ORDER BY segment ASC
LIMIT 5
"""